In [7]:
# download the spotify_db here: https://dbhub.io/aniketgope00/spotify_db.sqlite3
# move the spotify_db.sqlite3 to the same directory as this file

import sqlite3 as sql
import pandas as pd

connector = sql.connect("spotify_db.sqlite3")
cursor = connector.cursor()
query = f"select * from Data"
cursor.execute(query)
rows = cursor.fetchall()

data = pd.read_sql_query(query, connector)
data.head()

,index,idx,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,...,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3
1,1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,...,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4
2,2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,...,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4
3,3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,...,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4
4,4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,...,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4


In [13]:
def make_json_serializable(input_dict):
    def serialize_value(value):
        if isinstance(value, list):
            # Recursively process lists
            return [serialize_value(v) for v in value]
        elif isinstance(value, dict):
            # Recursively process dictionaries
            return {k: serialize_value(v) for k, v in value.items()}
        elif isinstance(value, (np.float32, np.float64)):
            return float(value)
        elif isinstance(value, (np.int32, np.int64)):
            return int(value)
        elif isinstance(value, (np.ndarray,)):
            # Convert numpy arrays to lists
            return value.tolist()
        return value  # Return as is if already JSON-serializable

    return {key: serialize_value(value) for key, value in input_dict.items()}

In [24]:
import os
import numpy as np
import spotdl
import torchaudio
import load_dotenv
import supabase

load_dotenv.load_dotenv()
url = os.getenv("project_url")
api_key = os.getenv("api_key_supabase")
supabase_client = supabase.create_client(url, api_key)


OUTPUT_FOLDER = "spotify_tracks"


def push_to_db(row):
    pass


def download_spotify_tracks(track_url, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    command = f"spotdl {track_url} --format wav --output \"{output_folder}"    
    os.system(command)


def extract_mfccs(file_path, n_mfcc=12):
    waveform, sample_rate = torchaudio.load(file_path, normalize=True)
    transform = torchaudio.transforms.MFCC(sample_rate=sample_rate,
                                           n_mfcc=20,
                                           melkwargs={"n_fft": 400, "hop_length": 160, "n_mels": 23, "center": False},)
    mfcc = transform(waveform)
    return mfcc

In [25]:
url = 'https://open.spotify.com/track/'
for i, row in data[:5].iterrows():
    track_url = url + row['track_id']
    download_spotify_tracks(track_url, OUTPUT_FOLDER)
    file_path = f"{OUTPUT_FOLDER}/{row['artist_name']} - {row['track_name']}.wav"
    mfccs = extract_mfccs(file_path)
    if mfccs is not None:
        print(f"Extracted MFCCs for {file_path}")
        row["mfcc"] = mfccs.tolist()  # Convert tensor to list
    else:
        print(f"Failed to extract MFCCs for {file_path}")
        continue

    row_dict = row.to_dict()
    row_json_serializable = make_json_serializable(row_dict)
    #push row to db
    supabase_client.table('songs_data').insert(row_json_serializable).execute()

    #delete file from OUTPUT_FOLDER
    os.remove(file_path)


Extracted MFCCs for spotify_tracks/Jason Mraz - I Won't Give Up.wav


ReadTimeout: The read operation timed out